In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [3]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [4]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [5]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2022-05-26 21:00:00+00:00,160.203,160.537,159.905,160.307
1,2022-05-27 01:00:00+00:00,160.305,160.637,160.138,160.458
2,2022-05-27 05:00:00+00:00,160.456,160.658,160.149,160.308
3,2022-05-27 09:00:00+00:00,160.307,160.562,159.826,160.381
4,2022-05-27 13:00:00+00:00,160.382,160.572,160.164,160.290


In [6]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [7]:
df_plot = df_ma.iloc[:500]

In [8]:
cp = CandlePlot(df_plot)

In [9]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [10]:
cp.show_plot(line_traces=traces)

In [11]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [12]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [13]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2022-07-13 01:00:00+00:00,162.668,163.408,162.668,163.108,163.13588,164.247815
1,2022-07-13 05:00:00+00:00,163.108,163.726,162.882,163.090,163.09272,164.261730
2,2022-07-13 09:00:00+00:00,163.086,163.714,162.860,163.119,163.08062,164.275035
3,2022-07-13 13:00:00+00:00,163.117,164.126,163.001,163.590,163.07276,164.291445
4,2022-07-13 17:00:00+00:00,163.588,163.650,163.265,163.420,163.09256,164.306640


In [14]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [15]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [16]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [17]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [18]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
51,2022-07-25 13:00:00+00:00,164.820,165.093,164.292,164.740,164.69278,164.651470,0.041310,-0.005960,1
78,2022-08-01 01:00:00+00:00,161.767,161.832,161.046,161.542,164.36944,164.414380,-0.044940,0.035605,-1
245,2022-09-07 21:00:00+00:00,165.785,166.325,165.761,166.083,162.52420,162.482745,0.041455,-0.039300,1
317,2022-09-25 21:00:00+00:00,155.138,155.640,149.333,149.497,162.73530,162.746095,-0.010795,0.256970,-1
386,2022-10-11 09:00:00+00:00,160.909,161.766,160.778,161.505,162.13024,162.123840,0.006400,-0.101630,1


In [19]:
cp = CandlePlot(df_an.iloc[220:260])
cp.show_plot(line_traces=[MA_S, MA_L])

In [20]:
ic.LoadInstruments("../data")

In [21]:
ic.instruments_dict[pair]

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 3}

In [22]:
ins_data = ic.instruments_dict[pair]

In [23]:
df_trades.shape

(22, 10)

In [24]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [25]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [26]:
df_trades.GAIN.sum()

np.float64(-2376.099999999994)

In [27]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [28]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])